In [1]:
!python3.11 -m pip install seaborn matplotlib tensorflow scikit-learn numpy


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import time
import os
import numpy as np
import gc
from sklearn.metrics import classification_report, mean_squared_error
import psutil

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

2025-03-29 17:52:29.058104: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-29 17:52:29.092590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-29 17:52:29.092675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-29 17:52:29.093674: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-29 17:52:29.099671: I tensorflow/core/platform/cpu_feature_guar

In [4]:
def read_csv_file(file_path):
    df = pd.read_csv(file_path)
    return df

# Load the dataset
# Assuming 'chunked_pbp_with_scores_2022.csv' is in the same directory
# If the file is in a different directory, provide the full path to the file
def group_by_game_id_and_chunk(df):
    grouped = df.groupby(['game_id', 'time_chunk'])
    return grouped 

df = read_csv_file('chunked_pbp_with_scores_2022.csv')
grouped_df = group_by_game_id_and_chunk(df)


def clean_scores(chunk_df):
    df = chunk_df.copy()
    
    df = df.sort_values('eventnum').reset_index(drop=True)
        
    # Initialize previous score and scoremargin
    prev_score = None
    prev_scoremargin = None
    for index, row in enumerate(df.itertuples(index=False)):
        # print(row.score, row.scoremargin, pd.isna(row.score), pd.isna(row.scoremargin))
        if row.score == '' or pd.isna(row.score):
            if prev_score is not None:
                # print(f"Using previous score: {prev_score}")
                df.at[index, 'score'] = prev_score
            else:
                # print("Both previous score and current score are None. Setting to default.")
                df.at[index, 'score'] = '0 - 0'
        else:
            # print(f"Current score is valid: {row.score}")
            prev_score = row.score

        if row.scoremargin == '' or pd.isna(row.scoremargin) or row.scoremargin == 'TIE':
            if prev_scoremargin is not None:
                # print(f"Using previous scoremargin: {prev_scoremargin}")
                df.at[index, 'scoremargin'] = prev_scoremargin
            else:
                # print("Both previous scoremargin and current scoremargin are None. Setting to default.")
                df.at[index, 'scoremargin'] = '0'
        else:
            # print(f"Current scoremargin is valid: {row.scoremargin}")
            prev_scoremargin = row.scoremargin
    # print("Final scores and score margins after filling:")
    # print(df[['score', 'scoremargin']])
    return df

        
        
    # # Replace 'TIE' with 0 and clean numeric scoremargin
    # df['scoremargin_clean'] = df['scoremargin'].replace({'TIE': '0', '': np.nan})
    # df['scoremargin_clean'] = pd.to_numeric(df['scoremargin_clean'], errors='coerce')

    # # Forward fill scoremargin
    # df['scoremargin_filled'] = df['scoremargin_clean'].ffill().fillna(0.0)

    # # Forward fill score (like '2 - 0')
    # df['score_filled'] = df['score'].ffill().fillna('0 - 0')

    # # Compute margin from score if score exists
    # def margin_from_score(score_str):
    #     try:
    #         home, away = map(int, score_str.strip().split('-'))
    #         return home - away
    #     except:
    #         return np.nan

    # df['margin_from_score'] = df['score_filled'].apply(margin_from_score)

    # # Final fallback: use margin from score if scoremargin was missing
    # df['scoremargin_final'] = df['scoremargin_clean'].combine_first(df['margin_from_score'])
    # df['scoremargin_final'] = df['scoremargin_final'].ffill().fillna(0.0)

    # return df[['eventnum', 'score', 'score_filled', 'scoremargin', 'scoremargin_filled', 'scoremargin_final']]


In [5]:
# Pick one sample group (first one)
sample_key = list(grouped_df.groups.keys())[0]
print(f"\nSample key: {sample_key}")

# Extract the chunk DataFrame
# For demonstration, we will take the first group as a sample

# Print the number of groups
chunk_df = grouped_df.get_group(sample_key)

cleaned_scores = clean_scores(chunk_df)
print("\nCleaned scores for the sample chunk:")
print(cleaned_scores["eventmsgactiontype"].value_counts())  # Display the count of each event type
print(cleaned_scores[['eventnum', 'score', 'scoremargin']])  # Display first 10 rows for brevity


Sample key: (22100001, 1.0)

Cleaned scores for the sample chunk:
eventmsgactiontype
0      15
1       4
79      4
72      3
11      2
2       2
12      2
3       2
52      1
80      1
47      1
5       1
97      1
103     1
Name: count, dtype: int64
    eventnum   score scoremargin
0          2   0 - 0           0
1          4   0 - 0           0
2          7   0 - 0           0
3          8   0 - 0           0
4          9   0 - 0           0
5         11   0 - 0           0
6         12   0 - 0           0
7         13   0 - 0           0
8         14   0 - 0           0
9         15   0 - 0           0
10        16   0 - 0           0
11        18   0 - 0           0
12        19   0 - 0           0
13        20   2 - 0          -2
14        22   2 - 0          -2
15        23   2 - 0          -2
16        24   2 - 2          -2
17        25   2 - 2          -2
18        26   2 - 2          -2
19        27   2 - 5           3
20        29   4 - 5           1
21        31   4 - 5  

In [6]:

event_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
event_encoder.fit(df[['eventmsgtype', 'eventmsgactiontype']])

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [7]:
print(event_encoder.categories_)

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 18]), array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  25,  26,  27,
        28,  29,  30,  33,  35,  36,  37,  39,  40,  41,  42,  43,  44,
        45,  47,  50,  51,  52,  57,  58,  63,  66,  67,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  86,  87,  93,  96,  97,  98,  99,
       100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110])]


In [8]:
def parse_score(score_str):
    try:
        parts = str(score_str).split(" - ")
        return [int(parts[0]), int(parts[1])]
    except:
        return [0, 0]

def parse_scoremargin(margin):
    if pd.isna(margin) or margin == 'TIE':
        return 0
    try:
        return int(float(margin))
    except:
        return 0

def vectorize_chunk(chunk_df, event_encoder, max_seq_len=100):
    """
    Vectorizes a chunk_df into a fixed-size 2D numpy array of shape (max_seq_len, feature_dim).
    Uses:
      - One-hot encoding for eventmsgtype and eventmsgactiontype
      - Parsed score and scoremargin
      - Numeric columns: period, eventnum, game_seconds, pts_home, pts_away, video_available_flag
    """
    # print(f"Input parameters are: {chunk_df.shape}, {event_encoder}, {max_seq_len}")
    if chunk_df.empty:
        print("Chunk is empty. Returning zero vector.")
        return np.zeros((max_seq_len, event_encoder.transform([[0, 0]]).shape[1] + 9))  # 2 from score, 1 from margin, 6 numerics

    # --- One-hot encoding of eventmsgtype + eventmsgactiontype ---
    # print(f"Before one-hot encoding, chunk_df has shape: {chunk_df.shape} and {chunk_df[['eventmsgtype', 'eventmsgactiontype']].dropna().shape}")
    event_ohe = event_encoder.transform(chunk_df[['eventmsgtype', 'eventmsgactiontype']])
    # print(f"One-hot encoded shape: {event_ohe.shape}")
    # print(f"Event OHE sample:\n{event_ohe[:5]}")  # Display first 5 rows for debugging

    # --- Score: "98 - 97" -> [98, 97] ---
    # print(f"Before parsing scores, chunk_df has shape: {chunk_df.shape} and score column:\n{chunk_df['score']}")
    score_matrix = np.array(chunk_df['score'].apply(parse_score).to_list())  # (N, 2)
    # print(f"After parsing scores, score_matrix shape: {score_matrix.shape}, sample:\n{score_matrix[:5]}")  # Display first 5 rows for debugging

    # --- Scoremargin: 'TIE' or NaN -> 0 ---
    # print(f"Before parsing scoremargin, chunk_df has shape: {chunk_df.shape} and scoremargin column:\n{chunk_df['scoremargin']}")
    scoremargin = chunk_df['scoremargin'].apply(parse_scoremargin).astype(float).values.reshape(-1, 1)
    # print(f"After parsing scoremargin, scoremargin shape: {scoremargin.shape}, sample:\n{scoremargin[:5]}")  # Display first 5 rows for debugging

    # --- Other numerical columns ---
    numeric_cols = ['period', 'eventnum', 'game_seconds']
    # print("Before extracting numeric columns, chunk_df has shape with numeric columns: ", chunk_df[numeric_cols].shape)   
    numeric_data = chunk_df[numeric_cols].fillna(0).astype(float).values  # (N, 6)
    # print("After extracting numeric columns, chunk_df has shape with numeric columns: ", chunk_df[numeric_cols].shape)   
    # --- Combine all features ---
    full_vector = np.concatenate([event_ohe, score_matrix, scoremargin, numeric_data], axis=1)  # (N, F)
    # print(f"Full vector shape after concatenation: {full_vector.shape}")
    # print(f"Full vector sample:\n{full_vector[:5]}")  # Display first 5 rows for debugging
    # --- Pad or truncate to max_seq_len ---
    seq_len, feature_dim = full_vector.shape

    if seq_len < max_seq_len:
        pad = np.zeros((max_seq_len - seq_len, feature_dim))
        full_vector = np.vstack([full_vector, pad])
    elif seq_len > max_seq_len:
        full_vector = full_vector[:max_seq_len]

    return full_vector  




chunk_df = clean_scores(grouped_df.get_group(sample_key))
# print("\nVectorizing chunk..., shape: ", chunk_df.shape)
# print("Columns in chunk_df: ", chunk_df.columns)
vectorized_chunk = vectorize_chunk(chunk_df, event_encoder)
# print("\nVectorized chunk shape: ", vectorized_chunk.shape)
# print("Vectorized chunk sample (first 5 rows):\n", vectorized_chunk[:5])  # Display first 5 rows for debugging
# print("\nVectorized chunk:\n", vectorized_chunk)

flattened_vector = vectorized_chunk.flatten()
print("\nFlattened vector shape: ", flattened_vector.shape)


Flattened vector shape:  (9600,)


In [9]:
from collections import defaultdict

group_cache = defaultdict(dict)
for (gid, chunk), group in grouped_df:
    group_cache[gid][chunk] = group

vector_cache = {}
for gid in group_cache:
    for chunk_id, chunk_df in group_cache[gid].items():
        cleaned = clean_scores(chunk_df)
        vector_cache[(gid, chunk_id)] = vectorize_chunk(cleaned, event_encoder)

print(group_cache)
print(vector_cache)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
X = []
y_cls = []   # For classification (home win/loss)
y_reg = []   # For regression (score difference)
chunk_window = 3
for game_id, game_df in df.groupby('game_id'):
        game_df = game_df.sort_values('time_chunk')
        chunks = sorted(game_df['time_chunk'].unique())
        if len(chunks) != 13:
            # this is for something that can't even get final label and have 3 chunk windows
            continue 
        final_chunk = chunks[-1]
        final_chunk_df = grouped_df.get_group((game_id, final_chunk))
        final_chunk_df = clean_scores(final_chunk_df)
        pts_home = final_chunk_df['pts_home'].iloc[-1]
        pts_away = final_chunk_df['pts_away'].iloc[-1]
        final_score = f"{pts_home} - {pts_away}"
        final_margin = pts_home - pts_away
        cls_label = 1 if final_margin > 0 else 0
        
        for i in range(0, len(chunks) - chunk_window):
            try:
                chunk_group = []
                valid_group = True
                for t in range(i, i + chunk_window):
                    vector = vector_cache.get((game_id, chunks[t]))
                    if vector is None:
                        valid_group = False 
                        break
                    chunk_group.append(vector)
            except KeyError:
                continue
            if not valid_group:
                continue   
            chunk_stack = np.stack(chunk_group)
            chunk_flat = chunk_stack.flatten()
            X.append(chunk_flat)
            y_cls.append(cls_label)
            y_reg.append(final_margin)

X = np.array(X)
y_cls = np.array(y_cls)
y_reg = np.array(y_reg)
            
print(f"\n✅ Created {len(X)} samples.")
print("X shape: ", X.shape)
print("y_cls shape: ", y_cls.shape)
print("y_reg shape: ", y_reg.shape)


✅ Created 11380 samples.
X shape:  (11380, 28800)
y_cls shape:  (11380,)
y_reg shape:  (11380,)


In [11]:
print(X.shape)

(11380, 28800)


In [12]:


# Scale input first (always before PCA)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reduce to 300 or 100 dims (try tuning)
pca = PCA(n_components=300)
X_reduced = pca.fit_transform(X_scaled)

print("New shape:", X_reduced.shape)  # (10*num_games, 300)

New shape: (11380, 300)


In [13]:


X_cls_train, X_cls_val, y_cls_train, y_cls_val = train_test_split(X_reduced, y_cls, test_size=0.2, random_state=42, stratify=y_cls)

In [14]:
X_reg_train, X_reg_val, y_reg_train, y_reg_val = train_test_split(X_reduced, y_reg, test_size=0.2, random_state=42, stratify=y_cls)

In [15]:
tf.keras.backend.clear_session()
gc.collect()

2345

In [18]:
from tensorflow.keras.callbacks import Callback

In [20]:
# RAM usage function
def print_memory_usage(stage=""):
    process = psutil.Process(os.getpid())
    mem_bytes = process.memory_info().rss
    mem_mb = mem_bytes / (1024 ** 2)
    print(f"[{stage}] RAM usage: {mem_mb:.2f} MB")

# Timing callback to monitor epoch durations
class TimingCallback(Callback):
    def on_train_begin(self, logs=None):
        self.times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_time_start
        self.times.append(epoch_time)
        print(f"Epoch {epoch+1} took {epoch_time:.2f} seconds")

print_memory_usage("After imports")

[After imports] RAM usage: 10082.58 MB


In [22]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_cls_train.shape[1],), kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

timing_callback = TimingCallback()

print_memory_usage("After model definition")

[After model definition] RAM usage: 10085.61 MB


In [24]:
print("\n--- Starting Training ---")
train_start = time.time()

history = model.fit(
    X_cls_train, y_cls_train,
    validation_data=(X_cls_val, y_cls_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop, timing_callback],
    verbose=1
)

train_end = time.time()
print(f"✅ Total training time: {train_end - train_start:.2f} seconds")
print_memory_usage("After training")


--- Starting Training ---
Epoch 1/30
285/285 [==============================] - 1s 3ms/step - loss: 1.2758 - accuracy: 0.4990 - auc: 0.4925 - val_loss: 1.1046 - val_accuracy: 0.5189 - val_auc: 0.4965
Epoch 2/30
285/285 [==============================] - 0s 2ms/step - loss: 1.0723 - accuracy: 0.5570 - auc: 0.5633 - val_loss: 1.0566 - val_accuracy: 0.5277 - val_auc: 0.4806
Epoch 3/30
285/285 [==============================] - 0s 2ms/step - loss: 0.9948 - accuracy: 0.5792 - auc: 0.6065 - val_loss: 1.0024 - val_accuracy: 0.5246 - val_auc: 0.5043
Epoch 4/30
285/285 [==============================] - 0s 2ms/step - loss: 0.9333 - accuracy: 0.6147 - auc: 0.6496 - val_loss: 0.9699 - val_accuracy: 0.5290 - val_auc: 0.4938
Epoch 5/30
285/285 [==============================] - 0s 2ms/step - loss: 0.8843 - accuracy: 0.6260 - auc: 0.6741 - val_loss: 0.9551 - val_accuracy: 0.5026 - val_auc: 0.5010
Epoch 6/30
285/285 [==============================] - 0s 2ms/step - loss: 0.8502 - accuracy: 0.6410 - a

In [26]:
print("\n--- Evaluating Model ---")
eval_start = time.time()

loss, acc, auc = model.evaluate(X_cls_val, y_cls_val, verbose=0)
print(f"📈 Val Accuracy: {acc:.4f} | Val AUC: {auc:.4f}")

eval_end = time.time()
print(f"✅ Evaluation time: {eval_end - eval_start:.2f} seconds")



--- Evaluating Model ---
📈 Val Accuracy: 0.5145 | Val AUC: 0.5072
✅ Evaluation time: 0.10 seconds


In [28]:
print("\n--- Predicting ---")
pred_start = time.time()

y_pred = (model.predict(X_cls_val, verbose=0) > 0.5).astype(int)

pred_end = time.time()
print(f"✅ Prediction time: {pred_end - pred_start:.2f} seconds")
print_memory_usage("After evaluation")


# === Classification Report ===
print("\n--- Classification Report ---")
report_start = time.time()

print(classification_report(y_cls_val, y_pred))

report_end = time.time()
print(f"✅ Classification report time: {report_end - report_start:.2f} seconds")
print_memory_usage("After classification report")


--- Predicting ---
✅ Prediction time: 0.17 seconds
[After evaluation] RAM usage: 7809.93 MB

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.54      0.66      0.60      1234
           1       0.46      0.34      0.39      1042

    accuracy                           0.51      2276
   macro avg       0.50      0.50      0.49      2276
weighted avg       0.50      0.51      0.50      2276

✅ Classification report time: 0.01 seconds
[After classification report] RAM usage: 7809.93 MB


In [30]:
weights, biases = model.layers[0].get_weights()
print("First Dense Layer Weights Shape:", weights.shape)
print("First Dense Layer Biases Shape:", biases.shape)


First Dense Layer Weights Shape: (300, 256)
First Dense Layer Biases Shape: (256,)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=032938a2-8262-4fbb-9ab0-ef927092c39b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>